# PAP C Kelompok 6

Tugas Group Project 2 :Klasifikasi

Impelementasi dan Eksperimen Prediksi Jenis Kejahatan menggunakan Python untuk Data 
Kejahatan di Los Angeles 

Anggota Kelompok:

1. Arif (5026221104)
2. Jasmine (5026221107)
3. Dicky (5026221036)
4. Affan (5026221134)

## Pemilihan Data

Tujuan: Membagi data yang sudah disiapkan dalam TGP #1 menjadi data pelatihan dan data tes.

In [2]:
#importing dependable
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

# Load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [6]:
#Load dataset
crime = pd.read_csv('Cleaned_Crime_Data.csv')
crime.head()

,Date Rptd,AREA,AREA NAME,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,...,Geographic_Zone,Female,Male,Unknown gender,Adult Arrest,Adult Other,Invest Cont,Juv Arrest,Juv Other,UNK
0,03/01/2020 12:00:00 AM,7,Wilshire,1,510,VEHICLE - STOLEN,NaN,37,M,O,...,Zone 1,False,True,False,True,False,False,False,False,False
1,02/09/2020 12:00:00 AM,1,Central,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O,...,Zone 1,False,True,False,False,False,True,False,False,False
2,11/11/2020 12:00:00 AM,3,Southwest,1,480,BIKE - STOLEN,0344 1251,19,X,X,...,Zone 1,False,False,True,False,False,True,False,False,False
3,05/10/2023 12:00:00 AM,9,Van Nuys,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O,...,Zone 2,False,True,False,False,False,True,False,False,False
4,08/18/2022 12:00:00 AM,6,Hollywood,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H,...,Zone 1,False,True,False,False,False,True,False,False,False


In [7]:
# Periksa missing value
crime.isnull().sum()

Date Rptd               0
AREA                    0
AREA NAME               0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes            142255
Vict Age                0
Vict Sex                0
Vict Descent            0
Premis Cd              15
Premis Desc           576
Weapon Used Cd     607445
Weapon Desc        607445
Status                  1
Crm Cd 1               11
LOCATION                0
Cross Street       764739
LAT                     0
LON                     0
TIME OCC_str            0
DATETIME OCC            0
day_of_week             0
month                   0
Day_Night               0
Geographic_Zone         0
Female                  0
Male                    0
Unknown gender          0
Adult Arrest            0
Adult Other             0
Invest Cont             0
Juv Arrest              0
Juv Other               0
UNK                     0
dtype: int64

In [9]:
dt = crime.copy()

In [16]:
# isi missing value kolom weapon used dengan code 500 dan deskripsi UNKNOWN WEAPON/OTHER WEAPON
# nilai yang kosong dalam weapon used mungkin dikarenakan senjata yang digunakan pelaku tidak diketahui

dt.fillna({'Weapon Used Cd':500}, inplace=True)
dt.fillna({'Weapon Desc':'UNKNOWN WEAPON/OTHER WEAPON'}, inplace=True)

In [17]:
# isi missing value kolom mocodes dengan modus
mocodes_modus = dt['Mocodes'].value_counts().idxmax()
dt.fillna({'Mocodes':mocodes_modus}, inplace=True)
dt.isnull().sum()

Date Rptd               0
AREA                    0
AREA NAME               0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes                 0
Vict Age                0
Vict Sex                0
Vict Descent            0
Premis Cd              15
Premis Desc           576
Weapon Used Cd          0
Weapon Desc             0
Status                  1
Crm Cd 1               11
LOCATION                0
Cross Street       764739
LAT                     0
LON                     0
TIME OCC_str            0
DATETIME OCC            0
day_of_week             0
month                   0
Day_Night               0
Geographic_Zone         0
Female                  0
Male                    0
Unknown gender          0
Adult Arrest            0
Adult Other             0
Invest Cont             0
Juv Arrest              0
Juv Other               0
UNK                     0
dtype: int64

In [18]:
# drop kolom Cross Street
dt = dt.drop('Cross Street', axis=1)

In [19]:
# drop baris yang memiliki null value
dt.dropna(axis=0, inplace=True)
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907917 entries, 0 to 908503
Data columns (total 34 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Rptd        907917 non-null  object 
 1   AREA             907917 non-null  int64  
 2   AREA NAME        907917 non-null  object 
 3   Part 1-2         907917 non-null  int64  
 4   Crm Cd           907917 non-null  int64  
 5   Crm Cd Desc      907917 non-null  object 
 6   Mocodes          907917 non-null  object 
 7   Vict Age         907917 non-null  int64  
 8   Vict Sex         907917 non-null  object 
 9   Vict Descent     907917 non-null  object 
 10  Premis Cd        907917 non-null  float64
 11  Premis Desc      907917 non-null  object 
 12  Weapon Used Cd   907917 non-null  float64
 13  Weapon Desc      907917 non-null  object 
 14  Status           907917 non-null  object 
 15  Crm Cd 1         907917 non-null  float64
 16  LOCATION         907917 non-null  object 
 